In [9]:
import numpy as np
from scipy.signal import stft

In [25]:
data = np.load('datasets/OpenBCI_GUI-v5-meditation.npy')
print(data.shape[0])
data.shape[0] / 8
data = data.swapaxes(1, 0)

# domínio do tempo
data.shape

175952


(8, 175952)

In [41]:
_, _, freqs = stft(data, fs=250., nperseg=256)
freqs = freqs.reshape(8, 129*1376)

# domínio da frequência
freqs.shape

(8, 177504)

Tarefa:
- Realizar plots com matplotlib dos dados em ambos os domínios para cada um dos canais
- Fazer a média dos 8 eletrodos e realizar os plot novamente

Desafio:
- Obter o PSD ($mod(w)^2$) para os valores no domínio da frequência e plotar novamente